In [2]:
import sqlite3
import pandas as pd

# Create connection (in-memory database)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create employees table
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    department TEXT,
    salary REAL
)
""")

# Ask user for number of employees

n = int(input("How many employees do you want to add? "))

# Insert data interactively

for i in range(n):
    print(f"\nEnter details for Employee {i+1}:")
    name = input("Name: ")
    #print(f"{name}")
    dept = input("Department: ")
    #print(f"{dept}")
    salary = float(input("Salary: "))
    #print(f"{salary}")
    cursor.execute("INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)", 
                   (name, dept, salary))
    print(f"{cursor.execute('SELECT * FROM employees').fetchall()}")

conn.commit()

print("\n Data inserted successfully!\n")

# Simple SELECT

query = "SELECT * FROM employees"
df = pd.read_sql(query, conn)
print("=== All Employees ===")
print(df.to_string(index=False), "\n")

# Filtering

query = "SELECT * FROM employees WHERE department = 'Accounting'"
df = pd.read_sql(query, conn)
print("=== Employees in IT Department ===")
print(df.to_string(index=False), "\n")

# Ordering

query = "SELECT * FROM employees ORDER BY salary DESC"
df = pd.read_sql(query, conn)
print("=== Employees Ordered by Salary (High → Low) ===")
print(df.to_string(index=False), "\n")

# Limit

query = "SELECT * FROM employees LIMIT 3"
df = pd.read_sql(query, conn)
print("=== First 3 Employees ===")
print(df.to_string(index=False))



Enter details for Employee 1:
[(1, 'Jay', 'ITSM', 80000.0)]

Enter details for Employee 2:
[(1, 'Jay', 'ITSM', 80000.0), (2, 'Priyam', 'IT-Software', 50000.0)]

Enter details for Employee 3:
[(1, 'Jay', 'ITSM', 80000.0), (2, 'Priyam', 'IT-Software', 50000.0), (3, 'Manas', 'Accounting', 35000.0)]

 Data inserted successfully!

=== All Employees ===
 id   name  department  salary
  1    Jay        ITSM 80000.0
  2 Priyam IT-Software 50000.0
  3  Manas  Accounting 35000.0 

=== Employees in IT Department ===
 id  name department  salary
  3 Manas Accounting 35000.0 

=== Employees Ordered by Salary (High → Low) ===
 id   name  department  salary
  1    Jay        ITSM 80000.0
  2 Priyam IT-Software 50000.0
  3  Manas  Accounting 35000.0 

=== First 3 Employees ===
 id   name  department  salary
  1    Jay        ITSM 80000.0
  2 Priyam IT-Software 50000.0
  3  Manas  Accounting 35000.0


In [ ]:
import sqlite3
import pandas as pd

# Database file (persistent)
db_file = "employees.db"

# Create connection
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create employees table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    department TEXT,
    salary REAL
)
""")
conn.commit()

def view_employees():
    df = pd.read_sql("SELECT * FROM employees", conn)
    if df.empty:
        print("\n No employees found.\n")
    else:
        print("\n=== Employee Records ===")
        print(df.to_string(index=False), "\n")

def add_employee():
    name = input("Enter name: ").strip()
    dept = input("Enter department: ").strip()
    try:
        salary = float(input("Enter salary: ").strip())
    except ValueError:
        print("Invalid salary, must be a number.")
        return
    cursor.execute("INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)", 
                   (name, dept, salary))
    conn.commit()
    print("Employee added successfully!\n")

def modify_employee():
    try:
        emp_id = int(input("Enter Employee ID to update: ").strip())
    except ValueError:
        print("Invalid ID.\n")
        return
    
    cursor.execute("SELECT * FROM employees WHERE id=?", (emp_id,))
    if cursor.fetchone() is None:
        print("Employee not found.\n")
        return
    
    name = input("Enter new name (leave blank to keep current): ").strip()
    dept = input("Enter new department (leave blank to keep current): ").strip()
    salary = input("Enter new salary (leave blank to keep current): ").strip()
    
    if name:
        cursor.execute("UPDATE employees SET name=? WHERE id=?", (name, emp_id))
    if dept:
        cursor.execute("UPDATE employees SET department=? WHERE id=?", (dept, emp_id))
    if salary:
        try:
            cursor.execute("UPDATE employees SET salary=? WHERE id=?", (float(salary), emp_id))
        except ValueError:
            print("Invalid salary. Update skipped for salary.")
    
    conn.commit()
    print("Employee updated successfully!\n")

def delete_employee():
    try:
        emp_id = int(input("Enter Employee ID to delete: ").strip())
    except ValueError:
        print("Invalid ID.\n")
        return
    
    cursor.execute("SELECT * FROM employees WHERE id=?", (emp_id,))
    if cursor.fetchone() is None:
        print("Employee not found.\n")
        return
    
    confirm = input("Are you sure you want to delete this employee? (y/n): ").strip().lower()
    if confirm == "y":
        cursor.execute("DELETE FROM employees WHERE id=?", (emp_id,))
        conn.commit()
        print("Employee deleted successfully!\n")
    else:
        print("Delete cancelled.\n")

def search_employees():
    print("\n Search by:")
    print("1. ID")
    print("2. Name")
    print("3. Department")
    choice = input("Enter your choice (1-3): ").strip()
    
    if choice == "1":
        try:
            emp_id = int(input("Enter Employee ID: ").strip())
        except ValueError:
            print("Invalid ID.\n")
            return
        df = pd.read_sql("SELECT * FROM employees WHERE id=?", conn, params=(emp_id,))
    elif choice == "2":
        name = input("Enter Name (partial allowed): ").strip()
        df = pd.read_sql("SELECT * FROM employees WHERE name LIKE ?", conn, params=(f"%{name}%",))
    elif choice == "3":
        dept = input("Enter Department (partial allowed): ").strip()
        df = pd.read_sql("SELECT * FROM employees WHERE department LIKE ?", conn, params=(f"%{dept}%",))
    else:
        print("Invalid choice.")
        return
    
    if df.empty:
        print("\n No matching records found.\n")
    else:
        print("\n=== Search Results ===")
        print(df.to_string(index=False), "\n")

# Menu-driven loop
while True:
    print("\n===== Employee Database Menu =====")
    print("1. View Employees")
    print("2. Add Employee")
    print("3. Modify Employee")
    print("4. Delete Employee")
    print("5. Search Employees")
    print("6. Exit")
    
    choice = input("Enter your choice (1-6): ").strip()
    
    if choice == "1":
        view_employees()
    elif choice == "2":
        add_employee()
    elif choice == "3":
        modify_employee()
    elif choice == "4":
        delete_employee()
    elif choice == "5":
        search_employees()
    elif choice == "6":
        print("Exiting... All changes saved.")
        break
    else:
        print("Invalid choice. Please enter 1-6.")
